In [5]:
import random

import gymnasium as gym
import numpy as np

import tensorflow as tf

from keras import Model
from keras.layers import Dense

In [6]:
class DQN(Model):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()

        self.replay_memory = []

        self.dense1 = Dense(128, activation="relu", input_dim=state_size)
        self.dense2 = Dense(64, activation="relu")
        self.dense3 = Dense(32, activation="relu")
        self.dense4 = Dense(action_size, activation="linear")

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return x
    
    def remember(self, state, action, reward, next_state):
        self.replay_memory.append((state, action, reward, next_state))

In [7]:
def update_model(model: DQN):
    # 리플레이 버퍼 크기가 작으면 업데이트하지 않음
    if len(model.replay_memory) < 1000:
        return
    
    # 너무 많으면 리플레이 버퍼 pop
    if len(model.replay_memory) > 10000:
        del model.replay_memory[0]

    # # 메모리에서 랜덤 샘플링
    sample_size = 32
    samples = random.sample(model.replay_memory, sample_size)

    # 분할
    states, actions, rewards, next_states = zip(*samples)

    # numpy 배열로 변환
    states = np.array(states)
    actions = np.array(actions)
    rewards = np.array(rewards)
    next_states = np.array(next_states)
    
    # 모델 예측과 타겟 값 계산
    targets = model.call(states).numpy()  # type: ignore
    next_q_values = model.call(next_states).numpy() # type: ignore

    targets[np.arange(sample_size), actions] = rewards + 0.97 * np.max(next_q_values, axis=1)
    
    left_actions = np.where(actions > 0 , actions - 1, 0)
    right_actions = np.where(actions < 8 , actions + 1, 8)

    targets[np.arange(sample_size), left_actions] = rewards + 0.97 * np.max(next_q_values, axis=1)
    targets[np.arange(sample_size), right_actions] = rewards + 0.97 * np.max(next_q_values, axis=1)


    # 모델 업데이트
    with tf.GradientTape() as tape:
        loss = tf.keras.losses.mean_squared_error(targets, model.call(states))
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    

In [9]:
import os    
import matplotlib.pyplot as plt

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

def draw_qvalue(model, episode):
    thetas = np.arange(-np.pi, np.pi, 0.1)
    velocities = np.arange(-4, 4, 0.1)

    states = np.array([[np.cos(theta), np.sin(theta), velocity] for theta in thetas for velocity in velocities])
    z = model.call(states).numpy()
    z_np = np.array(z).reshape((len(thetas), len(velocities), 9))

    fig, ax = plt.subplots(2, 5, figsize=(20, 8))
    fig.suptitle("Episode: {}".format(episode))
    for i in range(9):
        ax[i//5][i%5].title.set_text("Action: {}".format(i))
        ax[i//5][i%5].imshow(z_np[:,:,i], cmap='hot', interpolation='nearest', clim=(0, 1))
        ax[i//5][i%5].set_xticks(np.arange(0, 81, 10))
        ax[i//5][i%5].set_yticks(np.arange(0, 64, 10))
        ax[i//5][i%5].set_xticklabels(np.arange(-4, 5, 1))
        ax[i//5][i%5].set_yticklabels(np.round(np.arange(-np.pi, np.pi+0.1, 2*np.pi/6),1))
        ax[i//5][i%5].set_xlabel("Velocity")
        ax[i//5][i%5].set_ylabel("Theta")
        ax[i//5][i%5].invert_yaxis()  # y축 변경
    ax[1][4].axis('off')  # 마지막 subplot은 빈 공간

    plt.tight_layout()
    plt.savefig("model8_image/episode_{}.png".format(episode))

In [14]:
# gravity = 10
# env = gym.make("Pendulum-v1", g=9.81, render_mode="human")
# env = gym.make("Pendulum-v1", g=gravity, render_mode="human")
env = gym.make("Pendulum-v1", render_mode="human")
model = DQN(3, 9)

In [1]:
for episode in range(1000):
    state, info = env.reset()
    terminated = False
    step = 0

    rewards = []

    nice_state = False

    while not terminated and step < 500:

        # 모델로 행동 예측
        action_table = model.call(np.array([state])).numpy()[0]  # type: ignore
        action_table = np.exp(action_table) / np.sum(np.exp(action_table))
        action = np.random.choice(9, p=action_table)

        # 행동 실행
        next_state, _, terminated, _, _ = env.step((action/2-2,))
        reward = 0
        
        if next_state[0] > 0.985 and abs(next_state[2]) < 0.1:
            print("Perfect")
            reward += 100

        elif next_state[0] > 0.9:
            if abs(next_state[2]) < 0.2:
                print("Great")
                reward += 20
            else:
                if abs(next_state[2]) < abs(state[2]):
                    print("Good")
                    reward += 10
                else:
                    print("Bad")
                    reward -= 10

        else:
            reward = next_state[0]

        # 리플레이 버퍼에 기억
        model.remember(state, action, reward, next_state)
        update_model(model)

        state = next_state
        step += 1

        rewards.append(reward)

    # if episode % 10 == 0:
    #     draw_qvalue(model, episode)
    
    print("Episode: {}, Steps: {}, Average Reward: {}".format(episode, step, sum(rewards) / len(rewards)))
    # print("Episode: {}, Steps: {}, Score: {:.2f}, Last Score: {:.2f}".format(episode, step, sum(rewards) / len(rewards), sum(rewards[-10:])/10))

env.close()

NameError: name 'env' is not defined

In [ ]:
model.save_weights('model9', save_format='tf')